In [ ]:
%pip install spacy nltk 

In [3]:
%pip list

Package            Version
------------------ -----------
accelerate         1.6.0
aiohappyeyeballs   2.6.1
aiohttp            3.11.14
aiosignal          1.3.2
annotated-types    0.7.0
appnope            0.1.4
asttokens          3.0.0
async-timeout      5.0.1
attrs              25.3.0
bitsandbytes       0.42.0
blinker            1.9.0
blis               1.3.0
braceexpand        0.1.7
catalogue          2.0.10
certifi            2024.8.30
charset-normalizer 3.4.0
click              8.1.7
cloudpathlib       0.21.0
comm               0.2.2
confection         0.1.5
contourpy          1.3.1
cycler             0.12.1
cymem              2.0.11
datasets           3.5.0
debugpy            1.8.13
decorator          5.2.1
dill               0.3.8
docker-pycreds     0.4.0
einops             0.8.1
einops-exts        0.0.4
exceptiongroup     1.2.2
executing          2.2.0
filelock           3.16.1
Flask              3.1.0
fonttools          4.57.0
frozenlist         1.5.0
fsspec             2024.10.

In [4]:
import re
import json
import spacy
import nltk
from nltk import CFG, ChartParser
from nltk.tokenize import word_tokenize


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/riyagandhi/Library/Python/3.10/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/riyagandhi/Library/Python/3.10/lib/python/site-packages/traitlets/c

In [10]:
# Download NLTK data if not already done

nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/riyagandhi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/riyagandhi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [11]:
import pandas as pd 

In [26]:
VLMresponses = pd.read_csv("VLM_responses_Baseline.csv")
VLMresponses.head()

,UserPrompt,AssistantResponse,JSON,Accurate,Why it Failed?
0,I want to view traffic updates.,The user wants to view traffic updates. To com...,"{\n ""task"":""view traffic updates"",\n ""butt...",TRUE,NaN
1,I want to change the contact name to Jane Do.,To change the contact name to Jane Do using th...,"{\n ""task"":""change contact name to Jane Do"",...",TRUE,NaN
2,I want to add a red bag to my bag.,"Based on the image, the user would need to fol...",NaN,FALSE,NaN
3,I want to access my lessons offline.,"Based on the image, the user can complete the ...",NaN,FALSE,NaN
4,I want to give my answer verbally.,The image shows a mobile device displaying a u...,"{\n ""task"":""answer verbally and submit"",\n ...",TRUE,NaN


In [27]:
VLMresponses = VLMresponses[VLMresponses["Accurate"] != "FALSE"]

In [28]:
VLMresponses.head()

,UserPrompt,AssistantResponse,JSON,Accurate,Why it Failed?
0,I want to view traffic updates.,The user wants to view traffic updates. To com...,"{\n ""task"":""view traffic updates"",\n ""butt...",TRUE,NaN
1,I want to change the contact name to Jane Do.,To change the contact name to Jane Do using th...,"{\n ""task"":""change contact name to Jane Do"",...",TRUE,NaN
4,I want to give my answer verbally.,The image shows a mobile device displaying a u...,"{\n ""task"":""answer verbally and submit"",\n ...",TRUE,NaN
5,Can I maintain my current weight?,"Based on the image, to complete the task of 'C...","{\n ""task"":""maintain current weight"",\n ""b...",TRUE,NaN
6,Can you save my location?,"To save the user's location, the user should f...","{\n ""task"":""save location"",\n ""buttons"":[\...",TRUE,NaN


In [29]:
VLMresponses = VLMresponses.drop("Why it Failed?", axis='columns')
VLMresponses = VLMresponses.reset_index(drop=True)
VLMresponses.head()

,UserPrompt,AssistantResponse,JSON,Accurate
0,I want to view traffic updates.,The user wants to view traffic updates. To com...,"{\n ""task"":""view traffic updates"",\n ""butt...",TRUE
1,I want to change the contact name to Jane Do.,To change the contact name to Jane Do using th...,"{\n ""task"":""change contact name to Jane Do"",...",TRUE
2,I want to give my answer verbally.,The image shows a mobile device displaying a u...,"{\n ""task"":""answer verbally and submit"",\n ...",TRUE
3,Can I maintain my current weight?,"Based on the image, to complete the task of 'C...","{\n ""task"":""maintain current weight"",\n ""b...",TRUE
4,Can you save my location?,"To save the user's location, the user should f...","{\n ""task"":""save location"",\n ""buttons"":[\...",TRUE


In [49]:
# Sample Input
text = VLMresponses.iloc[1]["AssistantResponse"]
print(text)

jsonForm = VLMresponses.iloc[1]["JSON"]
print(jsonForm)


To change the contact name to Jane Do using the current screenshot, the user would need to follow these steps:

1. Tap on the 'Barack Obama' text field to edit the name.
2. Type 'Jane Do' in the text field.
3. Tap on the 'Save' button to confirm the changes.

Please note that the actual number of steps might be different depending on the app's design and functionality.
{
   "task":"change contact name to Jane Do",
   "buttons":[
      "Save"
   ],
   "input":[
      "text"
   ],
   "field":[
      "text"
   ]
}


In [43]:
def semantic_parse(text):
    task_match = re.search(r"To ([a-zA-Z\s]+@?)(?:.|\n|$)", text)
    buttons = re.findall(r'Tap on the\s+"([^"]+)"', text)

    return {
        "task": task_match.group(1) if task_match else None,
        "buttons": buttons
    }

In [44]:
print("\n=== Semantic Parser Output ===")
print(json.dumps(semantic_parse(text), indent=3))


=== Semantic Parser Output ===
{
   "task": "change the contact name to Jane Do using the current screenshot",
   "buttons": []
}


In [47]:
def improved_semantic_parser(text):
    if not isinstance(text, str):
        return None

    # Try different ways to extract the task
    task_patterns = [
        r"wants to ([a-zA-Z\s]+@?)(?:.|\n|$)",
        r"To ([a-zA-Z\s]+@?)(?:.|\n|$)",
        r"task of ['\"]?([a-zA-Z\s]+@?)['\"]?(?:.|\n|$)",
        r"complete the task of ['\"]?([a-zA-Z\s]+@?)['\"]?(?:.|\n|$)",
        r"user (?:can|should) ([a-zA-Z\s]+@?)(?:.|\n|$)"
    ]

    task = None
    for pattern in task_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            task = match.group(1).strip().rstrip(".")
            break

    # Extract quoted UI elements as buttons
    buttons = re.findall(r'"([^"]+@)"', text)
    buttons = [b.strip() for b in buttons if 1 <= len(b.strip().split()) <= 4]

    return {
        "task": task,
        "buttons": buttons
    }

In [48]:
print("\n=== Semantic Parser Output ===")
print(json.dumps(improved_semantic_parser(text), indent=3))


=== Semantic Parser Output ===
{
   "task": "change the contact name to Jane Do using the current screenshot",
   "buttons": []
}
